# VAE란?

## Variational AutoEncoder(변분추론 인코더)

입력데이터 X를 잘 설명하는 특징(feature)을 추출하여 Latent vector(잠재백터) Z에 담은후에, 이 Latent vector(잠재백터) Z를 통하여, X와 유사하나 완전 새로운 데이터를 생성하는것을 목표로합니다.

이때 각 특징(feature)이  가우시안 분포를 따른다고 가정하고, Latent vector(잠재백터)Z는 각 특징(feature)의 평균과 분산값을 나타냅니다.

VAE를 이용해 음악을 작곡한다고 가정한다면, 악기1,악기2,악기3의 속도, 피치의 평균 과 분산등의 정보등을 (feature)를 Latent vector(잠재백터) Z에 담고, 그 Latent vector(잠재백터) Z를 이용해 음악을 작곡 해 나가는 것 입니다.


### Latent vector(잠재백터)

Latent vector(잠재백터)는 독립적인 잠재 변수들의 쌍을 Latent vector(잠재백터)라고 부르며, GAN(적대적 생성신경망)등의 컴퓨터비전 분야에서도 사용합니다.




 

 입력 음원(midi데이터) X를 Variational AutoEncoder(변분추론 인코더)에 통과시켜, Latent vector(잠재백터) Z를 다시 Decoder에 통과시켜  입력 음원(midi데이터) X와 비슷하지만 새로운 데이터를 찾아내는 구조입니다.

Variational AutoEncoder(변분추론 인코더)는  입력 음원(midi데이터) X가 들어오면 그 데이터의 다양한 특징을 찾아 각각의 확률 변수가 되는 확률 분표를 만들게 됩니다.
이런 확률 분포를 잘 찾아, 값이 높은 부분을 이용해 실제 있을법한 음원(midi데이터)를 만들어 냅니다.

# 구현을 위한 패키지 설치


### |erorr note| dependency 이슈: magenta 2.1.0버전 설치하면됨.

!pip install magenta==2.1.0

In [1]:
#!pip install magenta==2.1.0

# 전처리 계획


- midi데이터를 학습하기위해, 백터화 해야함.

- magenta의 convert_directory 이용하여 디렉터리 채로 전처리

- zip파일로 다운받고 zipfile 라이브러리로 압축해제 후, 백터화

- 백터화하여 TFrecord로 저장

데이터URL:https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip


## TFrecord란

TFRecord 파일은 tensorflow의 학습 데이터등을 저장하기 위한 이진 데이터 포맷으로, 구글의 Protocol Buffer 포맷으로 데이터를 파일에 직렬화하여 저장합니다.

### 필요성

- 이미지 데이터와같은 데이터는 메타데이터와 레이블이 별도의 파일로 저장되어 있기때문에, 각각 읽어들여야해서 코드가 복잡해집니다.TFrecord파일을 이용하면 훨씬 간단하게 해결 할 수 있습니다.

- miidi 포멧으로 읽어서 매번 디코딩을 진행하면 학습단계에서 데이터를 읽을때 성능저하가 발생합니다. 이를 해결하기위해 TFrecord파일을 사용할 수 있습니다.

출저[https://bcho.tistory.com/1190]

# 전처리 소스코드

In [1]:
#=== 필요 module ===
import tensorflow as tf
import numpy as np
import pathlib #경로를 객체로 처리하기위해
import zipfile #zip파일 압축 해제를 위해
import os
import pandas as pd
import IPython
import collections
import note_seq #시퀀스 midi화 https://github.com/magenta/note-seq

from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import MusicVAE
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import data
from magenta.scripts.convert_dir_to_note_sequences import convert_directory #전처리
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel #훈련된모델
import tensorflow.compat.v1 as tf
import tf_slim 
#TF-Slim은 저수준의 텐서플로우 API를 간편하게 사용할 수 있는 고수준 경량 API

/Users/imjisu/anaconda3/envs/magenta/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/imjisu/anaconda3/envs/magenta/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/imjisu/anaconda3/envs/magenta/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywi

In [2]:
# #데이터 zip파일 불러오기
# url = "https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip"
# dir = tf.keras.utils.get_file(origin=url, 
#                                    fname='/Users/imjisu/content/data_dir/data.zip', 
#                                    extract=True)
# data_dir = pathlib.Path(dir)


In [3]:
#zip파일 압축해제
#zipfile.ZipFile('/content/data_dir/data.zip').extractall()

In [ ]:
#경로지정
data_root= '../groove' # 데이터 경로
csv_file = 'info.csv' # midi파일의 session, id, bpm 정보를 담고있음.
tfrec_root = 'data_dir/music.tfrecord'# tfrecord파일 경로를 지정

### info.csv 시각화

In [ ]:
import pandas as pd
df = pd.read_csv('groove/info.csv')
df = pd.DataFrame(df)
df.head(3)

### 전처리 진행(TFrecord화)

In [6]:
convert_directory(data_root,tfrec_root,recursive=True) #전처리 함수

INFO:tensorflow:Converting files in 'groove/'.
INFO:tensorflow:0 files converted.
INFO:tensorflow:Converting files in 'groove/drummer8'.
INFO:tensorflow:Converting files in 'groove/drummer8/session2'.
INFO:tensorflow:Converted MIDI file groove/drummer8/session2/12_funk_81_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file groove/drummer8/session2/25_latin_84_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file groove/drummer8/session2/2_funk_92_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file groove/drummer8/session2/41_highlife_126_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file groove/drummer8/session2/40_rock-indie_104_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file groove/drummer8/session2/33_rock_117_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file groove/drummer8/session2/30_afrobeat_98_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file groove/drummer8/session2/39_rock-indie_104_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file groove/drummer8/session2/14_afrobeat_103_beat_4-4.m

# config

- magenta/blob/main/magenta/models/music_vae/configs.py 수정
- 모델과 함께 전체 config를 수정
- tfrecord 경로지정 

# 모델 구조

### Encoder: BidirectionalLSTM(양방향 LSTM)

- 2계층 양방향 LSTM 

 역방향(미래에서 과거) 또는 정방향(과거에서 미래) 두 양방향으로
 시퀀스 정보를 갖도록 만듬.
 
 BidirectionalLSTM 을 사용하면 미래와 과거의 정보를 보전하기위해 양방향으로
 입력흐름을 만들수 있음.
 
 - NLP모델인 BERT에서도 사용


### 디코더: Hierarchical Decoder(GrooveLSTM)

- Simple RNN 사용 하면 기울기소실로 성능저하
 하위 시퀀스 간의 복잡한 종속성을 갖는 계층구조
 생성 프로세스를 모델링 하기위해 다양한 시간단계에 걸쳐, 잠재 확률 변수를 사용한
 신경망 기반 생성 아키텍처
 
- 역시 NLP모델에서 많이사용

## config 정의

In [7]:
#configs.py 스크립트 일부 발췌

class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

    def values(self):
        return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)


def update_config(config, update_dict):
    config_dict = config.values()
    config_dict.update(update_dict)
    return Config(**config_dict)


CONFIG_MAP = {}


HParams = contrib_training.HParams

# 모델 config configs.py의 groovae config 사용

#=== 모델구조 ===
CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(), #BidirectionalLstmEncoder 사용 
                   lstm_models.GrooveLstmDecoder()), #Hierarchical Decoder
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512, #데이터 배치사이즈
            max_seq_len=16 * 4,  # 4마디 길이지정
            z_size=256, #잠재백터 사이즈
            enc_rnn_size=[512], #인코더 순환 사이즈지정
            dec_rnn_size=[256, 256],#디코더 순환사이즈 지정
            max_beta=0.2,
            free_bits=48,
            dropout_keep_prob=0.3,#드롭아웃
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,

        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    # tfds_name='groove/4bar-midionly',
    train_examples_path='data_dir/music.tfrecord', #데이터 경로 설정
)

## 훈련정의 소스코드 

In [18]:
#============ License=====================================================
# Copyright 2022 The Magenta Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#=======================================================================

#=== MusicVAE train script===
#=== magenta/models/music_vae/music_vae_train.py ===

# Should not be called from within the graph to avoid redundant summaries.
def _trial_summary(hparams, examples_path, output_dir):
  #=== 텐서보드 summary 텍스트===

    examples_path_summary = tf.summary.text(
      'examples_path', tf.constant(examples_path, name='examples_path'),
      collections=[])

    hparams_dict = hparams.values()

#=== 하이퍼 파라미터===
  # Create a markdown table from hparams.
    header = '| Key | Value |\n| :--- | :--- |\n'
    keys = sorted(hparams_dict.keys())
    lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
    hparams_table = header + '\n'.join(lines) + '\n'

    hparam_summary = tf.summary.text(
      'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

    with tf.Session() as sess:
        writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
        writer.add_summary(examples_path_summary.eval())
        writer.add_summary(hparam_summary.eval())
        writer.close()


def _get_input_tensors(dataset, config):
  #== 데이터로부터 텐서 입력 ===
    batch_size = config.hparams.batch_size
    iterator = tf.data.make_one_shot_iterator(dataset)
    (input_sequence, output_sequence, control_sequence,
    sequence_length) = iterator.get_next()
    input_sequence.set_shape(
    [batch_size, None, config.data_converter.input_depth])
    output_sequence.set_shape(
    [batch_size, None, config.data_converter.output_depth])
    
    if not config.data_converter.control_depth:
        control_sequence = None
    
    else:
        control_sequence.set_shape(
            [batch_size, None, config.data_converter.control_depth])
        sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())
        
    return {
        'input_sequence': input_sequence,
        'output_sequence': output_sequence,
        'control_sequence': control_sequence,
        'sequence_length': sequence_length
    }

#=== 훈련 체크포인트, 시간 설정===
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):

#==== train loop ====
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)

    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(
            num_ps_tasks, merge_devices=True)):
            
            model = config.model
            model.build(config.hparams,
                        config.data_converter.output_depth,
                        is_training=True)
            #== 옵티마이저 ===
            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,num_sync_workers)
                hooks.append(optimizer.make_session_run_hook(is_chief))

            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            tf.summary.scalar('global_norm', global_norm)
            
            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                        grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads])
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
            train_op = optimizer.apply_gradients(
                list(zip(clipped_grads, var_list)),
                global_step=model.global_step,
                name='train_step')
            logging_dict = {'global_step': model.global_step,
                            'loss': model.loss}
            
            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))
                
            scaffold = tf.train.Scaffold(
                saver=tf.train.Saver(
                    max_to_keep=checkpoints_to_keep,
                    keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
            
            tf_slim.training.train(
                train_op=train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief)


def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator,
        is_training=True):
    config = config_map['groovae_4bar']
    train_dir = '.train'
    num_steps = 50000 #훈련 epoch
    
    def dataset_fn():
        return data.get_dataset(
            config,
            tf_file_reader=tf_file_reader,
            is_training=True,
            cache_dataset=True)
    
    if is_training == True:
        train(
            train_dir,
            config=config,
            dataset_fn=dataset_fn,
            num_steps=num_steps)      
    
    else:
        print("EVAL")


In [19]:
CONFIG_MAP

{'groovae_4bar': Config(model=<magenta.models.music_vae.base_model.MusicVAE object at 0x7fe2843bed90>, hparams=HParams([('batch_size', 512), ('beta_rate', 0.0), ('clip_mode', 'global_norm'), ('conditional', True), ('control_preprocessing_rnn_size', [256]), ('dec_rnn_size', [256, 256]), ('decay_rate', 0.9999), ('dropout_keep_prob', 0.3), ('enc_rnn_size', [512]), ('free_bits', 48), ('grad_clip', 1.0), ('grad_norm_clip_to_zero', 10000), ('learning_rate', 0.001), ('max_beta', 0.2), ('max_seq_len', 64), ('min_learning_rate', 1e-05), ('residual_decoder', False), ('residual_encoder', False), ('sampling_rate', 0.0), ('sampling_schedule', 'constant'), ('use_cudnn', False), ('z_size', 256)]), note_sequence_augmenter=None, data_converter=<magenta.models.music_vae.data.GrooveConverter object at 0x7fe2843b2150>, train_examples_path='data_dir/music.tfrecord', eval_examples_path=None, tfds_name=None)}

## 훈련 RUN

In [ ]:
run(CONFIG_MAP) #epoch 50000

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 512, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

INFO:tensorflow:
Decoder Cells:
  units: [256, 256]

INFO:tensorflow:Reading examples from file: data_dir/music.tfrecord
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


2024-07-17 14:43:21.277408: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into .train/model.ckpt.
INFO:tensorflow:.train/model.ckpt-0.data-00000-of-00001
INFO:tensorflow:49000
INFO:tensorflow:.train/model.ckpt-0.index
INFO:tensorflow:49000
INFO:tensorflow:.train/model.ckpt-0.meta
INFO:tensorflow:51700
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...


2024-07-17 14:43:42.744214: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 3021 of 5120
2024-07-17 14:43:48.822508: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


INFO:tensorflow:global_step = 0, loss = 568.56976
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 8...
INFO:tensorflow:Saving checkpoints for 8 into .train/model.ckpt.
INFO:tensorflow:.train/model.ckpt-8.data-00000-of-00001
INFO:tensorflow:49000
INFO:tensorflow:.train/model.ckpt-8.meta
INFO:tensorflow:51700
INFO:tensorflow:.train/model.ckpt-8.index
INFO:tensorflow:51700
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 8...
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 22...
INFO:tensorflow:Saving checkpoints for 22 into .train/model.ckpt.
INFO:tensorflow:.train/model.ckpt-22.index
INFO:tensorflow:0
INFO:tensorflow:.train/model.ckpt-22.meta
INFO:tensorflow:2700
INFO:tensorflow:.train/model.ckpt-22.data-00000-of-00001
INFO:tensorflow:51700
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 22...
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 35...
INFO:tensorflow:Saving checkpoints 

# 생성하기

In [ ]:
#=== magenta/magenta/models/music_vae/music_vae_generate.py ===
model = TrainedModel(
    config=CONFIG_MAP['groovae_4bar'],
    batch_size=1,
    checkpoint_dir_or_path='train') # 체크포인트의 경로

generated_sequence = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(generated_sequence[0], '/content/gen_midi/drum_4bar.mid')